# Exploring Applied NLP Problems

## Lecture Overview

* Named Entity Recognition (NER) using SpaCy and Transformers
* Text summarization using Transformers
* Text generation using Transformers
* Analyzing `Fake news` using Transformers and ChatGPT
* Semantic role labeling using Transformers and ChatGPT

## Named Entity Recognition (NER) using SpaCy and Transformers

### Named Entity Recognition (NER)

Named Entity Recognition (NER) is the task of identifying named entities in text and classifying them into pre-defined categories such as the names of persons, organizations, locations, expressions of times, quantities, monetary values, percentages, etc.

### SpaCy Example

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(f'Entity: {ent.text} is type {ent.label_} - index_location: {ent.start_char}:{ent.end_char}')

In [ ]:
### more complex data

# https://pubmed.ncbi.nlm.nih.gov/37071411/
text = """
Most patients were initially certified for a 1:1 (∆9-tetrahydrocannabinol:cannabidiol) tincture.
Eight-seven percent of patients (n = 60) were noted to exhibit an improvement in any PD symptom after starting MC.
Symptoms with the highest incidence of improvement included cramping/dystonia, pain, spasticity, lack of appetite, dyskinesia, and tremor.
After starting MC, 56% of opioid users (n = 14) were able to decrease or discontinue opioid use with an average daily morphine milligram equivalent change from 31 at baseline to 22 at the last follow-up visit.
The MC was well-tolerated with no severe AEs reported and low rate of MC discontinuation due to AEs (n = 4).
"""

doc = nlp(text)

for ent in doc.ents:
    print(f'Entity: {ent.text} is type {ent.label_} - index_location: {ent.start_char}:{ent.end_char}')

In [ ]:
## Using SciSpacy

import spacy
import scispacy

nlp = spacy.load("en_core_sci_scibert")

doc = nlp(text)

for ent in doc.ents:
    print(f'Entity: {ent.text} is type {ent.label_} - index_location: {ent.start_char}:{ent.end_char}')

```
Entity: patients is type ENTITY - index_location: 6:14
Entity: ∆9-tetrahydrocannabinol:cannabidiol) tincture is type ENTITY - index_location: 51:96
Entity: patients is type ENTITY - index_location: 121:129
Entity: improvement is type ENTITY - index_location: 164:175
Entity: PD is type ENTITY - index_location: 183:185
Entity: symptom is type ENTITY - index_location: 186:193
Entity: MC is type ENTITY - index_location: 209:211
Entity: Symptoms is type ENTITY - index_location: 213:221
Entity: incidence is type ENTITY - index_location: 239:248
Entity: improvement is type ENTITY - index_location: 252:263
Entity: cramping/dystonia is type ENTITY - index_location: 273:290
Entity: pain is type ENTITY - index_location: 292:296
Entity: spasticity is type ENTITY - index_location: 298:308
Entity: lack of appetite is type ENTITY - index_location: 310:326
Entity: dyskinesia is type ENTITY - index_location: 328:338
Entity: tremor is type ENTITY - index_location: 344:350
Entity: MC is type ENTITY - index_location: 367:369
Entity: opioid is type ENTITY - index_location: 378:384
Entity: users is type ENTITY - index_location: 385:390
Entity: decrease is type ENTITY - index_location: 413:421
Entity: discontinue is type ENTITY - index_location: 425:436
Entity: opioid is type ENTITY - index_location: 437:443
Entity: daily is type ENTITY - index_location: 464:469
Entity: morphine is type ENTITY - index_location: 470:478
Entity: milligram is type ENTITY - index_location: 479:488
...
Entity: low rate is type ENTITY - index_location: 620:628
Entity: MC is type ENTITY - index_location: 632:634
Entity: discontinuation is type ENTITY - index_location: 635:650
Entity: AEs is type ENTITY - index_location: 658:661
```

### Transformers Example

In [ ]:
import transformers
from transformers import pipeline

ner = pipeline('ner', model='dslim/bert-base-NER', tokenizer='dslim/bert-base-NER', grouped_entities=True)
ner(text)

### Using a different model

In [ ]:
# https://huggingface.co/d4data/biomedical-ner-all
ner = pipeline('ner', model='d4data/biomedical-ner-all', tokenizer='d4data/biomedical-ner-all', grouped_entities=True)
ner(text)

### Creating your own pipeline

* Extract semantic triples from the text then perform NER on the extracted triples
* Use Stanford CoreNLP to extract semantic triples from the text then perform NER on the extracted triples

In [ ]:
import stanza
import spacy
from stanza.server import CoreNLPClient
stanza.install_corenlp()

## extract triples from the text
triples = []

# define the properties
config = {
    "annotators": "tokenize,ssplit,pos,lemma,ner,parse,depparse,coref,openie",
    "openie.max_entailments_per_clause": "100",
    "openie.threads": "4",
    "memory": "16G",
    "endpoint": "http://localhost:9020",
}

client = CoreNLPClient(annotators=config['annotators'], memory=config['memory'], endpoint=config['endpoint'])

document = client.annotate(text)
for i, sentence in enumerate(document.sentence):
    for triple in sentence.openieTriple:
        triples.append([triple.subject, triple.relation, triple.object])
        
triples

In [ ]:
from IPython.display import HTML, display

display(HTML(text))

In [ ]:
## Analyze the triples for NER

import spacy

nlp = spacy.load("en_core_web_sm")

for i, triple in enumerate(triples):
    doc = " ".join(triple)
    doc = nlp(doc)
    for ent in doc.ents:
        print(f'Entity: {ent.text} is type {ent.label_} - index_location: {ent.start_char}:{ent.end_char}')

In [ ]:
ner = pipeline('ner', model='dslim/bert-base-NER', tokenizer='dslim/bert-base-NER', grouped_entities=True)

for i, triple in enumerate(triples):
    doc = " ".join(triple)
    doc = ner(doc)
    for ent in doc:
        print(f'Entity: {ent["word"]} is type {ent["entity_group"]} - index_location: {ent["start"]}:{ent["end"]}')

## Text summarization using Transformers

There are two types of text summarization:

* Extractive summarization: Extracting a subset of the original text to form the summary
* Abstractive summarization: Generating new text to form the summary

### Extractive summarization

There are several extractive summarization techniques:

* LexRank - LexRank is a graph-based algorithm that uses the PageRank algorithm to rank sentences based on their similarity to other sentences in the text.
* SentRank - SentRank is a graph-based algorithm that uses the PageRank algorithm to rank sentences based on their similarity to other sentences in the text.
* Luhn - Uses TF-IDF to rank sentences based on their similarity to other sentences in the text.
* SumBasic - Utilize the frequency of words in the text to rank sentences. (abstract-like)
* KL-Sum - Kullback-Leibler divergence is used to rank sentences based on their similarity to other sentences in the text.
* LSA - Latent semantic analysis or indexing uses singular value decomposition to compute matrices for analyzing relationships between sets of observations.
* K-Means - K-means clustering is a method of vector quantization, originally from signal processing, that is popular for cluster analysis in data mining.

### Abstractive summarization

In [ ]:
import transformers

from transformers import BloomTokenizerFast
from transformers import BloomForCausalLM

MODEL = BloomForCausalLM.from_pretrained('bigscience/bloom-560m')
TOKENIZER = BloomTokenizerFast.from_pretrained('bigscience/bloom-560m')

In [ ]:
## summarization of our text using the Bloom model

def summarize_text(text: str, tokenizer=TOKENIZER, min_output=40, max_output=100, max_length=80, model=MODEL):
    """Take a string of text and generate a summary"""
    tokens_input = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length=max_length, truncation=True)
    ids = model.generate(tokens_input, min_length=min_output, max_length=max_output)
    summary = tokenizer.decode(ids[0], skip_special_tokens=True)
    return summary

In [ ]:
summary = summarize_text(text)
display(HTML(summary))

## Fake News

What is fake news?

* Fake news is a type of yellow journalism or propaganda that consists of deliberate misinformation or hoaxes spread via traditional print and broadcast news media or online social media.
* Fake news can be published to intentionally or circumstantially damage the reputation of a person or entity, or make money through advertising revenue.
* But ... fake news is not always false. The label can be used to discredit news that is critical of a person or organization, or to draw attention away from critical analysis.

### Fake News Detection processing

* Is it a news article?
* Is there consensus on the truthfulness of the article?
* If yes, return the consensus
* If no, continue
  * What is challenged in the article?
    * Sentiment analysis - can shed light on the overall tone of the article (positive, negative, neutral) - heatmap of the article by paragraph or section
    * Named entity recognition - can we identify the entities in the article (people, places, organizations, etc.)
    * Can we perform semantic role labeling on the article?
    * Are there references to other sources?



adapted from Rothman, D. _Transformers for Natural Language Processing_. O'Reilly Media, Inc., 2020

## Exploring Document Similarity

### Using vectors to analyze sentence or document similarity

#### Dot product

The dot product or inner product of two vectors is defined as:

$$ \vec{a} \cdot \vec{b} = \sum_{i=1}^{n} a_i b_i $$


With our vectors are defined as:

$$ |v| = \sqrt{\sum^{N}_{i=1} v^2_i}$$

* The longer the vector, the larger the magnitude
* More frequent words will have larger magnitude
* Raw dot product is not normalized - how can we use it to measure similarity?


#### Normalized dot product

$$ \vec{a} \cdot \vec{b} = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum^{N}_{i=1} a^2_i} \sqrt{\sum^{N}_{i=1} b^2_i}} $$

#### Cosine similarity

With the cosine similarity, we can measure the angle between two vectors. The cosine similarity is defined as:

$$ \text{cosine(a, b)  = } \frac{a\cdot b}{|a||b|} = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum^{N}_{i=1} a^2_i} \sqrt{\sum^{N}_{i=1} b^2_i}} $$

### Cosine similarity of words

https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin

### Cosine similarity of sentences and documents

How can we use the word vectors to measure the similarity between sentences or documents?

* Average the word vectors in the sentence or document
* Calculate the cosine similarity between the two sentences or documents
* Train a classifier to predict the similarity between sentences or documents
* Train a sentence embedding model to generate sentence or document vectors
* etc.

#### Doc2Vec

https://radimrehurek.com/gensim/models/doc2vec.html

In [ ]:
import gensim

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
Doc2Vec??

In [ ]:
common_texts

In [ ]:
model.infer_vector(["system", "response"])

### Let's train a Doc2Vec model

In [ ]:
import os
import re
import pandas as pd
from gensim import corpora, models, similarities

df = pd.read_csv('../datasets/news-2023-02-01.csv')

df.head()

In [ ]:
titles = df['title'].tolist()
titles = [title for title in titles if type(title) == str]

In [ ]:
dictionary = corpora.Dictionary([title.split() for title in titles])

In [ ]:
dictionary.token2id

In [ ]:
# Convert the reviews into Gensim bag-of-words vectors
corpus = [dictionary.doc2bow(title.split()) for title in titles]

In [ ]:
# Train a Gensim TF-IDF model on the corpus
tfidf = models.TfidfModel(corpus)

In [ ]:
# Convert the corpus into Gensim TF-IDF vectors
tfidf_corpus = tfidf[corpus]

In [ ]:
tfidf_corpus?

In [ ]:
# examine the first 10 documents
for doc in tfidf_corpus[:10]:
    print(doc)

In [ ]:
# Train a Gensim LSI model on the TF-IDF vectors
lsi_model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=25)

In [ ]:
lsi_model??

In [ ]:
lsi_model_topics = lsi_model.show_topics(formatted=False)
lsi_model_topics

In [ ]:
lsi_model.get_topics()

In [ ]:
lsi_model.print_topics(num_topics=10, num_words=5)